# Packages

In [16]:
# Import essential packages
import urllib
from urllib.parse import urlencode
import requests
import pandas as pd
import os.path, time
import hashlib

# Import functions
from ipynb.fs.full.functions import existing_file

### Temporary annotations
# https://api.semrush.com/?type=phrase_organic&key={{key}}&phrase=bouwmaterialen&export_columns=Dn,Ur,Fk,Fp&database=nl&display_limit=20
# https://api.semrush.com/?type=phrase_organic&key=b9fd485dbbcc0d31bc30e828806fd14c&phrase=bouwmaterialen&export_columns=Dn,Ur,Fk,Fp&database=nl&display_limit=5
# https://api.semrush.com/?type=phrase_organic&key=b9fd485dbbcc0d31bc30e828806fd14c&phrase=bouwmaterialen&export_columns=Dn,Ur,Ts&database=nl&display_limit=1

ImportError: cannot import name 'existing_file' from 'ipynb.fs.full.functions' (unknown location)

# Variables

In [13]:
### Below code is for keywords
endpoint_url = 'https://api.semrush.com/'
export_columns = 'Dn,Ur'
params = {
    "?type": 'phrase_organic',
    'key': 'b9fd485dbbcc0d31bc30e828806fd14c',
    'phrase': 'bouwmaterialen',
    'export_columns': export_columns, # Columns from https://www.semrush.com/api-analytics/#columns
    'database': 'nl', # Country of market
    'display_limit': '50', # Count of retrieving results
    }

# ### Below code is for backlinks
# endpoint_url = 'https://api.semrush.com/analytics/v1/'
# export_columns = 'external_num,internal_num'
# params = {
#     '?key': 'b9fd485dbbcc0d31bc30e828806fd14c',
#     'type': 'backlinks',
#     'target': 'bouwmaat.nl',
#     'target_type': 'root_domain',
#     'export_columns': export_columns,
#     'display_limit': '1'
#     }

### Do you want to overwrite the data if it already exists?
potential_overwrite = False

# Functions

In [14]:
### Function used to monitor credit use for SEMrush API
def semrush_call(params):
    data = urllib.parse.urlencode(params, doseq=True)
    main_call = urllib.parse.urljoin(endpoint_url, data)
    main_call = main_call.replace(r'%3F', r'?')
    return main_call

### Function used to parse data from semrush_call
def parse_response(call_data):
    results = []
    data = call_data.decode('unicode_escape')
    lines = data.split('\r\n')
    lines = list(filter(bool, lines))
    columns = lines[0].split(';')
        
    for line in lines[1:]:
        result = {}
        for i, datum in enumerate(line.split(';')):
            result[i] = datum.strip('"\n\r\t')
        results.append(result)
    return results

### Function used to rename columns to readable format
def rename_columns(finalKeywordData):
    x = export_columns.split(',')
    finalKeywordData = finalKeywordData.rename(columns={finalKeywordData.columns[0]:'Number'})

    for i in range(0, (len(finalKeywordData.columns)-1)):
        finalKeywordData = finalKeywordData.rename(columns={finalKeywordData.columns[i+1]:x[i]})
        
    return finalKeywordData
        
### Function used to call all other functions to get the data in a formatted way
def execute_call():
    params_hashed = hash_variable(params)
    file_exists = existing_file(params_hashed, "call")
    if file_exists == True: # Just load CSV file
        finalKeywordData = pd.read_csv(f"calls\call-{params_hashed}.csv")
        return rename_columns(finalKeywordData)
    else: # Execute query and save to CSV file
        data = requests.get(semrush_call(params))
        parsed_data = parse_response(data.content)
        finalKeywordData = pd.DataFrame(parsed_data)
        # Add the finalKeywordData to the dataframe and a new CSV file
        finalKeywordData.to_csv(f"calls\call-{params_hashed}.csv")
        finalKeywordData = pd.read_csv(f"calls\call-{params_hashed}.csv") # Make sure to read from CSV to easily make format uniform
        return rename_columns(finalKeywordData)

# Executing of functions

In [15]:
# Executing of final function
finalKeywordData = execute_call()
finalKeywordData

NameError: name 'hash_variable' is not defined

# Unused code

In [145]:
### Unusued for loop to use more keywords in a given query
# for every in yourKeywords:
#     data=requests.get(semrush_call(phrasetype, every, export_columns, database, display_limit))
#     parsed_data = parse_response(data.content)
#     df = pd.DataFrame(parsed_data)
#     keyword_frame.append(df)